In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import argparse
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
from time import time

In [2]:
def load_data(batch_size=128):
    """
    Download and load the data
    """
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    train_set = torchvision.datasets.CIFAR10(root='./data.cifar10', train=True,
                                            download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                              shuffle=True, num_workers=2)

    test_set = torchvision.datasets.CIFAR10(root='./data.cifar10', train=False,
                                           download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                             shuffle=False, num_workers=2)

    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    return train_loader, test_loader, classes

In [3]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, 5, stride=1)
        self.conv1_bn = torch.nn.BatchNorm2d(32)
        self.conv2 = torch.nn.Conv2d(32, 64, 3)
        self.conv2_bn = torch.nn.BatchNorm2d(64)
        self.conv3 = torch.nn.Conv2d(64, 128, 3)
        self.conv3_bn = torch.nn.BatchNorm2d(128)
        
        self.fc1 = torch.nn.Linear(128 * 2 * 2, 120)
        self.fc1_bn = torch.nn.BatchNorm1d(120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc2_bn = torch.nn.BatchNorm1d(84)
        self.fc3 = torch.nn.Linear(84, 10)
        
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.dropout1d = torch.nn.Dropout(0.3)
        self.dropout2d = torch.nn.Dropout2d(0.3)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1_bn(self.conv1(x))))
        x = self.dropout2d(x)
        x = self.pool(F.relu(self.conv2_bn(self.conv2(x))))
        x = self.dropout2d(x)
        x = self.pool(F.relu(self.conv3_bn(self.conv3(x))))
        x = self.dropout2d(x)
#         print(x.shape)
        x = x.view(-1, self.flatten_feature(x))
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = self.dropout1d(x)
        x = F.relu(self.fc2_bn(self.fc2(x)))
        x = self.dropout1d(x)
        x = self.fc3(x)
        
        return x
    
    def flatten_feature(self, x):
        num_feature = 1
        for d in x.size()[1:]:
            num_feature *= d
        return num_feature



In [4]:
def test(net, inputs, device, criterion=None, flag='test'):
    net.eval()
    if flag == 'inference':
        with torch.no_grad():
            image = inputs.to(device)
            output = net(image)
            _, predicted = torch.max(output.data, 1)
            return predicted.int()
    else:
        correct = 0
        total = 0
        total_loss = 0
        with torch.no_grad():
            for data in inputs:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total_loss += criterion(outputs, labels)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        return 100 * correct / total, float(total_loss/len(inputs))

In [7]:
writer = SummaryWriter('./runs/3_conv_drop_0.3_iter_500')

total_epoch = 500


train_loader, test_loader, classes = load_data()
net = Net()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# if torch.cuda.device_count() > 1:
#     print("Using", torch.cuda.device_count(), "GPUs" )
#     net = torch.nn.DataParallel(net)
net.to(device)
    
best_test_accuracy = 0
old_train_lost = 0
start = time()
for epoch in range(1, total_epoch+1):
    for i, data in enumerate(train_loader, 0):
        net.train()
        # get the inputs; data is a list of [inputs, labels]
        inputs = data[0].to(device)
        labels = data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    train_accuracy, train_loss = test(net, train_loader, device, criterion)
    test_accuracy, test_loss = test(net, test_loader, device, criterion)
    if epoch % 10 == 0:
        print("### Epoch [{}/{}]\t\ttrain accuracy: {}%\t\ttrain loss: {}\t\ttest accuracy: {}%\t\ttest loss:{}"
              .format(epoch,total_epoch, train_accuracy, round(train_loss, 7), test_accuracy, round(test_loss, 4)))
    writer.add_scalar('Test/Loss', test_loss, epoch)
    writer.add_scalar('Test/Accuracy', test_accuracy, epoch)
    writer.add_scalar('Train/Loss', train_loss, epoch)
    writer.add_scalar('Train/Accuracy', train_accuracy, epoch)


    if best_test_accuracy < test_accuracy:
        best_test_accuracy = test_accuracy
        if not os.path.isdir('./model'):
            os.mkdir('./model')
        torch.save(net.state_dict(), "./model/fc.pt")
        # print("### Epoch {}\t\tSave Model fot test accuracy {}% ".format(epoch+1, best_test_accuracy))
    if abs(old_train_lost-train_loss) < 1e-5:
        print("Training almost converge in Epoch {}, STOP!".format(epoch))
print("Finish in {}!".format(time()-start))
print("The best Model for testing accuracy of {}% is saving in ./model".format(best_test_accuracy))

Files already downloaded and verified
Files already downloaded and verified
### Epoch [10/500]		train accuracy: 70.544%		train loss: 0.8506929		test accuracy: 67.52%		test loss:0.9279
### Epoch [20/500]		train accuracy: 77.02%		train loss: 0.6803358		test accuracy: 72.41%		test loss:0.8077
### Epoch [30/500]		train accuracy: 80.11%		train loss: 0.5964293		test accuracy: 74.32%		test loss:0.7481
### Epoch [40/500]		train accuracy: 81.998%		train loss: 0.5445241		test accuracy: 75.21%		test loss:0.7213
### Epoch [50/500]		train accuracy: 83.492%		train loss: 0.5021561		test accuracy: 76.0%		test loss:0.697
### Epoch [60/500]		train accuracy: 84.596%		train loss: 0.46828		test accuracy: 76.7%		test loss:0.6805
### Epoch [70/500]		train accuracy: 85.66%		train loss: 0.4440726		test accuracy: 77.11%		test loss:0.6677
### Epoch [80/500]		train accuracy: 86.34%		train loss: 0.4241743		test accuracy: 77.45%		test loss:0.6544
### Epoch [90/500]		train accuracy: 86.904%		train loss: 0.4105394		t

In [23]:
images = train_loader.dataset[0]

In [32]:
np.array(images[0]).shape

(3, 32, 32)

In [34]:
from torch

'Net(\n  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))\n  (conv1_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))\n  (conv2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))\n  (conv3_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n  (fc1): Linear(in_features=512, out_features=120, bias=True)\n  (fc1_bn): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n  (fc2): Linear(in_features=120, out_features=84, bias=True)\n  (fc2_bn): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n  (fc3): Linear(in_features=84, out_features=10, bias=True)\n  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)\n  (dropout1d): Dropout(p=0.3, inplace=False)\n  (dropout2d): D

In [5]:
train_loader, test_loader, classes = load_data()

Files already downloaded and verified
Files already downloaded and verified


In [10]:
next(iter(train_loader))[1].shape

torch.Size([128])